In [5]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import hashlib

# Key derivation using PBKDF2
def derive_key(password, salt, key_length=32):
    return hashlib.pbkdf2_hmac('sha256', password.encode(), salt, 100000, dklen=key_length)

# Hybrid encryption function
def hybrid_encrypt(plaintext, password):
    salt = get_random_bytes(16)  # Generate a new salt for each encryption
    key = derive_key(password, salt, 32)  # Derive a 256-bit key
    iv = get_random_bytes(16)  # AES block size IV

    cipher = AES.new(key, AES.MODE_CBC, iv)  # AES in CBC mode
    padded_plaintext = pad(plaintext.encode(), AES.block_size)  # Apply PKCS7 padding
    ciphertext = cipher.encrypt(padded_plaintext)

    return salt + iv + ciphertext  # Return salt + IV + encrypted data

# Hybrid decryption function
def hybrid_decrypt(encrypted_data, password):
    salt = encrypted_data[:16]  # Extract salt
    iv = encrypted_data[16:32]  # Extract IV
    ciphertext = encrypted_data[32:]  # Extract encrypted message

    key = derive_key(password, salt, 32)  # Derive key using the same salt

    cipher = AES.new(key, AES.MODE_CBC, iv)  # AES in CBC mode
    decrypted_padded = cipher.decrypt(ciphertext)  # Decrypt data

    try:
        decrypted = unpad(decrypted_padded, AES.block_size).decode()  # Remove padding correctly
        return decrypted
    except ValueError:
        raise ValueError("Decryption failed: Incorrect padding or wrong password")

# Example usage
password = "securekey"
plaintext = "Hello"

encrypted = hybrid_encrypt(plaintext, password)
print("Encrypted:", encrypted.hex())

decrypted = hybrid_decrypt(encrypted, password)
print("Decrypted:", decrypted)


Encrypted: f42d0c6b4332519f754a8b4c899abf9fe59cae1b5d4b01a9a1b4dbf045db2616a317a9020038e5f2c81c658e52bf8764
Decrypted: Hello


In [2]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00
